In [153]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from scipy import stats
import warnings
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mean 
import re
import glob
from statsmodels.stats.multitest import multipletests
!pip install xlsxwriter
pd.options.mode.chained_assignment = None  # default='warn'

DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [154]:
def custom_sort(item):
    if item.endswith('basal'):
        return 0
    else:
        return int(item.split('uM')[0].split('_')[-1])

In [155]:
currdir= os.getcwd()
parent = os.path.dirname(currdir)
gparent=os.path.dirname(parent)
lines_to_skip = 10 # adjust

EXPERIMENT_TIME = "1_MIN"
plate_type = 'DIV19'

PERTURBATION = "CNO"
firstTableHeading = "Number of Spikes"

FOLDER_PATH = f"{parent}/example_data/Div19_CNO/Div19_1minute" #
folder_dict = {}
balances= {"basal": "",
        "dose1": "",
        "dose2": "",
        "dose3": "",
        "dose4": ""
        }
## WALK Through folder of interest
for dirpath, dirnames, filenames in os.walk(FOLDER_PATH):
    if dirpath != FOLDER_PATH:
        folder_dict[dirpath.split("/")[-1]] = dirpath
    elif not dirnames:
        folder_dict['basal'] = dirpath

## get doses names in increasing order 
dose_names = [name for name in folder_dict.keys() if 'basal' not in name]
dose_names.sort(key=custom_sort)

# Initialize doses dictionaryd
balances = {"basal": "basal"}

## map sorted doses
for i, dose_name in enumerate(dose_names, start=1):
    balances[f"dose{i}"] = dose_name.split("_")[-1]

# Output the updated doses dictionary
print("Balances:", balances)
folder_dict

Balances: {'basal': 'basal', 'dose1': '5uMCNO', 'dose2': '10uMCNO', 'dose3': '15uMCNO', 'dose4': '20uMCNO'}


{'Div19_5uMCNO': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeArrayAnalysis/example_data/Div19_CNO/Div19_1minute/Div19_5uMCNO',
 'Div19_10uMCNO': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeArrayAnalysis/example_data/Div19_CNO/Div19_1minute/Div19_10uMCNO',
 'Div19_basal': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeArrayAnalysis/example_data/Div19_CNO/Div19_1minute/Div19_basal',
 'Div19_20uMCNO': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeArrayAnalysis/example_data/Div19_CNO/Div19_1minute/Div19_20uMCNO',
 'Div19_15uMCNO': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeArrayAnalysis/example_data/Div19_CNO/Div19_1minute/Div19_15uMCNO'}

In [156]:
def read_csvs_to_dict(folder, lines_to_skip=10):
    files = glob.glob(os.path.join(folder, "*.csv"))
    data_dict = {}
    for file in files:
        # Extracting the plate number from the filename
        plate_num = re.search(r'plate\s?(\d+)', file, re.IGNORECASE)
        if plate_num:
            plate_key = f"plate_{plate_num.group(1)}"
            df = pd.read_csv(file, skiprows = lines_to_skip)
            df = df.drop(columns=['Unnamed: 9'])
            data_dict[plate_key] = df
    return data_dict
all_files_dict = {}

for dose, folder in folder_dict.items():
    all_files_dict[dose] = read_csvs_to_dict(folder)
    
for dose, files_dict in all_files_dict.items():
    print(dose, files_dict.keys())

Div19_5uMCNO dict_keys(['plate_1', 'plate_3'])
Div19_10uMCNO dict_keys(['plate_1', 'plate_3'])
Div19_basal dict_keys(['plate_1', 'plate_3'])
Div19_20uMCNO dict_keys(['plate_3', 'plate_1'])
Div19_15uMCNO dict_keys(['plate_3', 'plate_1'])


In [157]:
balances

{'basal': 'basal',
 'dose1': '5uMCNO',
 'dose2': '10uMCNO',
 'dose3': '15uMCNO',
 'dose4': '20uMCNO'}

In [158]:
def concat_plates(plates_dict, firstHeading):
    experiment = {}
    testparams_concat = {}
    for plate, df in plates_dict.items():
        maxrows = df.shape[0]
        vals_between_tables = 16
        titles = [heading_ind for heading_ind in range(14, maxrows+16, 16)]
        titles.insert(0,0)
        # # Empty dictionary to store dataframes
        # Loop through start indices
        for ind in range(len(titles) -1):
            if ind == 0:
                ## Handle mean firing rate
                key = firstHeading
                table = df.iloc[titles[ind] : titles[ind +1], :]
                # table = rename_row_names(table)
            elif ind != 0:
                table = df.iloc[titles[ind] : titles[ind +1], :]
                key = table.iloc[0][0] # just get name
                table = df.iloc[titles[ind] +2: titles[ind +1], :]

            table.set_index('Unnamed: 0', inplace=True)
            if key not in experiment:
                experiment[key] = []
            experiment[key].append(table)
    
    for testparam, df_list in experiment.items():
        concated_table = pd.concat(df_list, ignore_index=False)
        testparams_concat[testparam] = concated_table
    
    testparams_concat = {key.strip(): value for key, value in testparams_concat.items()}
    return testparams_concat
doses = {}
for dose, plates in all_files_dict.items():
    doses[dose] = concat_plates(plates, firstTableHeading)

doses.keys()

dict_keys(['Div19_5uMCNO', 'Div19_10uMCNO', 'Div19_basal', 'Div19_20uMCNO', 'Div19_15uMCNO'])

In [159]:
def remove_mean_SEM(test_dict, chemical_type):
    for key, value in test_dict.items():
        to_drop = [index for index in ['B Mean', 'B SEM'] if index in value.index]
        if to_drop:
            test_dict[key] = value.drop(index=to_drop)
        test_dict[key]['Chemical_Type']=chemical_type
    return test_dict
for dose, parameter_dict in doses.items():
    doses[dose] = remove_mean_SEM(parameter_dict, dose)


In [160]:
def rename_row_names(experiment_df):
    # 'Unnamed: 0' is a column
    experiment_df = experiment_df.reset_index()
    experiment_df = experiment_df.drop(columns="Unnamed: 0", errors="ignore")
    experiment_df.index = ["Replicate{}".format(i+1) for i in range(len(experiment_df))]
    return experiment_df
for dose, test_dict in doses.items():
    for test_name, df in test_dict.items():
        test_dict[test_name] = rename_row_names(df)
    doses[dose] = test_dict

In [161]:
doses.keys()

dict_keys(['Div19_5uMCNO', 'Div19_10uMCNO', 'Div19_basal', 'Div19_20uMCNO', 'Div19_15uMCNO'])

In [162]:
balances.keys(), balances.values(), doses.keys()

(dict_keys(['basal', 'dose1', 'dose2', 'dose3', 'dose4']),
 dict_values(['basal', '5uMCNO', '10uMCNO', '15uMCNO', '20uMCNO']),
 dict_keys(['Div19_5uMCNO', 'Div19_10uMCNO', 'Div19_basal', 'Div19_20uMCNO', 'Div19_15uMCNO']))

In [163]:
dmapped = {}
for bkey, bval in balances.items():
    for dkey, dval in doses.items():
        if bval == dkey.split("_")[-1]:
            dmapped[bkey] = dval

list(map(lambda x: dmapped[x]['Mean Firing Rate (Hz)']['Chemical_Type'].unique(), dmapped.keys()))

[array(['Div19_basal'], dtype=object),
 array(['Div19_5uMCNO'], dtype=object),
 array(['Div19_10uMCNO'], dtype=object),
 array(['Div19_15uMCNO'], dtype=object),
 array(['Div19_20uMCNO'], dtype=object)]

In [164]:
for dose, tdict in dmapped.items():
    print(dose)

basal
dose1
dose2
dose3
dose4


In [165]:
def impute_nans(testparam, table):
    chem_types = table['Chemical_Type']
    table = table.drop(columns=['Chemical_Type'])
    original_index = table.index
    imputer = IterativeImputer(max_iter=100, random_state=0)
    imputed_data = imputer.fit_transform(table)

    # Convert the imputed data back to a DataFrame
    imputed_df = pd.DataFrame(imputed_data, columns=table.columns, index=original_index)
    imputed_df['Chemical_Type'] = chem_types
    return imputed_df
dmappedI = {}
for dose, tdict in dmapped.items():
    dmappedI[dose] = {testname: impute_nans(testname, table) for testname, table in tdict.items() if not table.loc[:, table.columns != 'Chemical_Type'].isna().all().all()} 


/home/poojaparameswaran/anaconda3/envs/anovaenv/lib/python3.8/site-packages/sklearn/impute/_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/home/poojaparameswaran/anaconda3/envs/anovaenv/lib/python3.8/site-packages/sklearn/impute/_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/home/poojaparameswaran/anaconda3/envs/anovaenv/lib/python3.8/site-packages/sklearn/impute/_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [173]:
def perform_t_test(condition_dict):
    tests = {}
    for testname in condition_dict.keys():
        tests[testname] = {}
        table = condition_dict[testname].copy()  # Create a deep copy
        rows_to_drop = [row for row in ['B Mean', 'B SEM', 'B Std'] if any(row.lower() == existing_row.lower() for existing_row in table.index)]
        if rows_to_drop:
            table.drop(rows_to_drop, axis=0, inplace=True)
        test_variables = [col for col in table.columns if "unt" not in col.lower() and "chemical" not in col.lower()]
        control_name = "unt2"
        for testitem in test_variables:
            exp_group = list(table[testitem].astype(float))
            control_group = list(table[control_name].astype(float))
            with warnings.catch_warnings(record=True) as w:
                warnings.simplefilter("always")  # Cause all warnings to always be triggered
                t_stat, pval = stats.ttest_ind(exp_group, control_group)
                fold_change = mean(exp_group)/ mean(control_group) if mean(control_group)!=0 else 1 # if control on top mutant.mean /wt.mean  fc >1 == expgroup downregulated fc <1 expgroup upregulated
                if len(w) > 0:
                    tests[testname][testitem] = [t_stat, pval, fold_change, 'Warning: ' + str(w[-1].message)]
                else:
                    tests[testname][testitem] = [t_stat, pval, fold_change]
    return tests

dmapped_ttest = {}
for key, testdict in dmappedI.items():
    dmapped_ttest[key] = perform_t_test(testdict)
dmapped_ttest.keys()

dict_keys(['basal', 'dose1', 'dose2', 'dose3', 'dose4'])

In [174]:
def adjust_for_multipletesting(one_dosedict):
    group_geneacrosstest = {}
    for test, genedict in one_dosedict.items():
        for gene, values in genedict.items():
            if gene not in group_geneacrosstest:
                group_geneacrosstest[gene] = []
            group_geneacrosstest[gene].append(values[1])
    return group_geneacrosstest

def multipletest(one_dosedict):
    pvals_grouped= adjust_for_multipletesting(one_dosedict)
    for gene, plst in pvals_grouped.items():
        hypothesis, pvals_adjust, alpha_sidak, alpha_bonfer = multipletests(plst, method='bonferroni')
        pvals_grouped[gene] = {}
        pvals_grouped[gene]["hypothesis"] = hypothesis
        pvals_grouped[gene]["pvalsadjust"] = pvals_adjust
        for ind, test in enumerate(one_dosedict.keys()):
            one_dosedict[test][gene][1] = pvals_adjust[ind]
            one_dosedict[test][gene].insert(2 ,hypothesis[ind])
    return one_dosedict

dmapped_ttesta = {}
for ind, (key, testdict) in enumerate(dmapped_ttest.items()):
    dmapped_ttesta[key] = multipletest(testdict)
dmapped_ttesta.keys()

dict_keys(['basal', 'dose1', 'dose2', 'dose3', 'dose4'])

In [175]:
dmapped_ttesta['basal']

{'Number of Spikes': {'GPR37L 1': [-0.7702999096628084,
   1.0,
   False,
   0.7809124803013413],
  'GPR37L 2': [-0.1485826488014328, 1.0, False, 0.9642925779298149],
  'LGI2 A12': [-1.0705636505874039, 1.0, False, 0.7427835645923819],
  'SLITRK5': [0.250447584250342, 1.0, False, 1.0679555675135486],
  'THSD7 1': [-1.7469565400986873, 1.0, False, 0.5863858246531114],
  'THSD7 2': [-1.4411930248795166, 1.0, False, 0.6508052427259099]},
 'Mean Firing Rate (Hz)': {'GPR37L 1': [-0.7702998432207158,
   1.0,
   False,
   0.7809124893193676],
  'GPR37L 2': [-0.1485824463388916, 1.0, False, 0.9642926255912573],
  'LGI2 A12': [-1.0705635265276336, 1.0, False, 0.7427835842686557],
  'SLITRK5': [0.25044775910194467, 1.0, False, 1.067955619637556],
  'THSD7 1': [-1.7469564186802684, 1.0, False, 0.5863858437464791],
  'THSD7 2': [-1.4411928620785492, 1.0, False, 0.6508052707422801]},
 'ISI Coefficient of Variation - Avg': {'GPR37L 1': [-0.20974793794422733,
   1.0,
   False,
   0.9728968901467764],

In [176]:
def ensure_dirs_exists(path):
    directory = os.path.dirname(path)  # Get the directory part of the file path
    if not os.path.exists(directory):
        os.makedirs(directory)
    return

In [170]:
# exp1.iloc[62:79]
def analyze_test_results(onedose_dict, outfile):
    # col_names = ['Test Types', 'Gene', 'T-statistic', 'Regulation (FC)', 'adj P-value', 'Significance', 'Warnings']
    df = pd.DataFrame()
    for test, gdict in (onedose_dict.items()):
        for prot in gdict:
            new_row = {}
            new_row['Test Types'] = test
            new_row['Gene'] = prot
            new_row['T-statistic (control.mean - mutant.mean)'] = gdict[prot][0] # + = downregulated, - = upregulated
            new_row['Regulation mutant.mean / control.mean'] = gdict[prot][-1] # >1 == downregulatrd, <1 upregulated
            pval = gdict[prot][1]
            new_row['P-value'] = pval
            
            new_row['Significance (hyp rej)'] = gdict[prot][2]
            if len(gdict[prot]) > 4:
                new_row['Warnings'] = gdict[prot][-1]
            else:
                new_row['Warnings'] = "No Warning"
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    ensure_dirs_exists(outfile)
    df.to_csv(outfile)
    print(f"written to {outfile}")
    return df

for dosage, dosedict in dmapped_ttest.items():
    test_results = analyze_test_results(dosedict, 
                                        f'{parent}/t-test_results/{FOLDER_PATH.split("/")[-2]}/{FOLDER_PATH.split("/")[-1]}/'
                                            f'{balances[dosage]}_ttest_results.csv')
    test_results

written to /home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeArrayAnalysis/t-test_results/Div19_CNO/Div19_1minute/basal_ttest_results.csv
written to /home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeArrayAnalysis/t-test_results/Div19_CNO/Div19_1minute/5uMCNO_ttest_results.csv
written to /home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeArrayAnalysis/t-test_results/Div19_CNO/Div19_1minute/10uMCNO_ttest_results.csv
written to /home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeArrayAnalysis/t-test_results/Div19_CNO/Div19_1minute/15uMCNO_ttest_results.csv
written to /home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeArrayAnalysis/t-test_results/Div19_CNO/Div19_1minute/20uMCNO_ttest_results.csv


In [171]:
dmapped_ttest.keys()

dict_keys(['basal', 'dose1', 'dose2', 'dose3', 'dose4'])

In [179]:
def save_all_to_excel(data, outfile):
    ensure_dirs_exists(outfile)
    with pd.ExcelWriter(outfile, engine='xlsxwriter') as writer:
        for dosage, dosedict in data.items():
            # Flatten the data structure
            flat_data = {}
            for test, gdict in dosedict.items():
                index = gdict.keys()
                flat_data[(test, 't-stat')] = [v[0] for v in gdict.values()]
                flat_data[(test, 'adj p-value')] = [v[1] for v in gdict.values()]
                flat_data[(test, 'FoldChange')] = [v[3] for v in gdict.values()]


            # Create DataFrame with MultiIndex columns
            df = pd.DataFrame(flat_data, index =index)            
            df.columns = pd.MultiIndex.from_tuples(df.columns)
            df.to_excel(writer, sheet_name=balances[dosage])
    writer.close()
    if os.path.exists(outfile):
        return f"Outfile created {outfile}"
    else:
        return f"Problem in writing conditions to {outfile}"

save_all_to_excel(dmapped_ttest,f'{parent}/t-test_results/{FOLDER_PATH.split("/")[-2]}/{FOLDER_PATH.split("/")[-1]}/'
                                            f'ttest_results_{FOLDER_PATH.split("/")[-2].split("_")[-1]}{FOLDER_PATH.split("/")[-1].split("_")[-1]}.xlsx')


/home/poojaparameswaran/anaconda3/envs/anovaenv/lib/python3.8/site-packages/xlsxwriter/workbook.py:368: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


'Outfile created /home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeArrayAnalysis/t-test_results/Div19_CNO/Div19_1minute/ttest_results_CNO1minute.xlsx'